In [3]:
import pandas as pd
import seaborn as sns
'''

df = pd.read_csv(
    'files/results_pct_20201103.txt', sep=' ',header=None, error_bad_lines=False, keep_default_na=False)
df.head()
'''
# shrav already did it

"\n\ndf = pd.read_csv(\n    'files/results_pct_20201103.txt', sep=' ',header=None, error_bad_lines=False, keep_default_na=False)\ndf.head()\n"

In [12]:
df = pd.read_fwf('files/voter_stats_20201103.txt', error_bad_lines=False)
df = df['county_desc\telection_date\tstats_type\tprecinct_abbrv\tvtd_abbrv\tparty_cd\trace_code']
df = df.str.split(pat = '\t')

df.head()

0    [IREDELL, 11/03/2020, voter, EM, EM, DEM, U, H...
1    [GUILFORD, 11/03/2020, voter, G60, G60, LIB, W...
2    [WAKE, 11/03/2020, voter, 07-12, 07-12, REP, W...
3    [MOORE, 11/03/2020, voter, RBN, RBN, UNA, U, U...
4    [WAKE, 11/03/2020, voter, 19-18, 19-10, UNA, A...
Name: county_desc\telection_date\tstats_type\tprecinct_abbrv\tvtd_abbrv\tparty_cd\trace_code, dtype: object